# **Convolutional Neural Networks**

**Disclaimer**: large parts of the lab are taken from [here](https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/).


**Definition**: convolutional neural networks (CNNs) are usually used for image recognition. A network is a convolutional network if it uses _convolution_ at least in one layer (the convolutional layer).

<img src="https://drive.google.com/uc?id=1C4-xQW2aCzNt2BKVEKH-aNVxeoZNkCjK"
 style="float:center;width:50px;height:50px;" align="center">




**The convolutional layer**: the convolutional layer _extracts features_ (sharpen, blur...) from the pixels. It is a mathematical operation between the input image and the kernel (filter).  

<img src="https://drive.google.com/uc?id=1iQeSoQvSAsWbvigpsafMSzVfw4G8_00m"
 style="float:center;width:50px;height:50px;" align="center">

 [Credits](https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/)




**The pool layer**: pooling layers reduce the dimensionality of the previous layer, mantaining the most important features. Common choices are the max and the average value.

<img src="https://drive.google.com/uc?id=1Lv0DkhlpSDbsGyze2qtmT53c-xQM969K"
 style="float:center;width:50px;height:50px;" align="center">


 [Credits](https://www.researchgate.net/publication/333593451_Application_of_Transfer_Learning_Using_Convolutional_Neural_Network_Method_for_Early_Detection_of_Terry%27s_Nail/figures?lo=1)

Let us try to build our first CNN!
As usual, first thing first, we import the libraries and we ask for GPUs.

In [ ]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Let us define some useful parameters!

In [ ]:
# parameters
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

Let us import the Dataset.
The below code:
* defines a transformation over the data (resizes, converts and normalizes),
* downloads the data,
* defines dataloaders which load the data in batches (the RAM does not suffer).



In [ ]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

We have to create the CNN and, as usual, we extend the ``nn.Module`` and define the layers in the constructor.

Then, we have to define the ``forward`` method. The input channel is 3 (3 channels: red, green and blue). We are "enlarging" the third dimension to captures more features (opacity, sharpness...).

Then we pool. You define how many pixel you want to consider (2x2) and the _stride_, i.e. how do you move along the pixels.

In the end, a fully connected layer is defined (nothing new).


In [ ]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

Let us define the loss and the optimizer.

In [ ]:
seed = 0
torch.manual_seed(seed)

model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

Let us train the model and test it!

In [ ]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))
